In [1]:
%matplotlib inline
import threading
import numpy as np
from numpy.linalg import norm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import ensemble, tree, linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from tqdm import tqdm

import progressbar
from importlib import reload

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, log_loss
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from gensim.models.doc2vec import TaggedDocument
from gensim import utils

c:\users\laomd\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [8]:
from nltk.corpus import wordnet as wn

In [18]:
words = set()
for sysset in wn.synsets("wonderful"):
    words.update(sysset.lemma_names())

In [19]:
words

{'fantastic',
 'grand',
 'howling',
 'marvellous',
 'marvelous',
 'rattling',
 'terrific',
 'tremendous',
 'wonderful',
 'wondrous'}

In [63]:
df = pd.read_excel('data/回归/train.xlsx')
tags = df['Tags'].values
del df['Tags']

labels = df['Reviewer_Score']
del df['Reviewer_Score']

In [68]:
all_tags = []
for i in range(len(tags)):
    all_tags.append(list(map(lambda x: x.strip("' '"), tags[i].strip('[] ').split(','))))

In [69]:
all_tags

[['Family with young children', 'Standard Queen Room', 'Stayed 1 night'],
 ['Leisure trip',
  'Family with young children',
  'Double Room with Courtyard View',
  'Stayed 5 nights',
  'Submitted from a mobile device'],
 ['Business trip', 'Couple', 'Superior Room', 'Stayed 1 night'],
 ['Leisure trip',
  'Couple',
  'Deluxe King Room with River View',
  'Stayed 1 night',
  'Submitted from a mobile device'],
 ['Leisure trip', 'Solo traveler', 'Deluxe Single Room', 'Stayed 1 night'],
 ['Leisure trip', 'Solo traveler', 'Double Room', 'Stayed 5 nights'],
 ['Business trip', 'Solo traveler', 'Deluxe King Room', 'Stayed 1 night'],
 ['Leisure trip',
  'Family with young children',
  'Queen Room with Sofa Bed Non Smoking',
  'Stayed 2 nights',
  'Submitted from a mobile device'],
 ['Business trip', 'Solo traveler', 'Classic Single Room', 'Stayed 3 nights'],
 ['Leisure trip',
  'Couple',
  'Standard Double or Twin Room',
  'Stayed 5 nights',
  'Submitted from a mobile device'],
 ['Leisure trip', '

In [74]:
def constructTaggedDocuments(data):
    res = []
    index = 0
    for row in data:
        for tag in row:
            res.append(TaggedDocument (tag.split(), ['Text' + '_%s' % str(index)]))
            index += 1
    return res

In [75]:
sentences = constructTaggedDocuments(all_tags)

In [76]:
sentences[:5]

[TaggedDocument(words=['Family', 'with', 'young', 'children'], tags=['Text_0']),
 TaggedDocument(words=['Standard', 'Queen', 'Room'], tags=['Text_1']),
 TaggedDocument(words=['Stayed', '1', 'night'], tags=['Text_2']),
 TaggedDocument(words=['Leisure', 'trip'], tags=['Text_3']),
 TaggedDocument(words=['Family', 'with', 'young', 'children'], tags=['Text_4'])]

In [77]:
from gensim.models import Doc2Vec

In [78]:
import os

Text_INPUT_DIM=15


text_model=None
filename='Doc2Vec-trainTags-' + str(Text_INPUT_DIM) + '.vec'
text_model = Doc2Vec(min_count=1, window=5, vector_size=Text_INPUT_DIM, sample=1e-4, negative=5, workers=4, epochs=25,seed=1)
text_model.build_vocab(sentences)
text_model.train(sentences, total_examples=text_model.corpus_count, epochs=text_model.epochs)
text_model.save(filename)

In [ ]:
text_modelt_model = Doc2Vec.load(filename)

In [96]:
tags_trainX = np.array([text_model.docvecs['Text_'+str(i)] for i in range(len(sentences))])

In [97]:
tags_trainX.shape

(365733, 15)

In [101]:
tags_trainX[:5]

array([[ 0.22147569,  0.00914983,  0.14314692,  0.18509114, -0.20429121,
        -0.0713853 ,  0.07562377,  0.08337325,  0.02333423,  0.0761978 ,
         0.00130978,  0.14169283,  0.09772088,  0.2250386 , -0.03409904],
       [ 0.03874396, -0.00775344,  0.16450088, -0.05136273,  0.04751065,
         0.17342319, -0.00619348,  0.01533997,  0.06941085, -0.07018629,
        -0.02164743,  0.08342565, -0.00756676,  0.07255357, -0.12217735],
       [-0.00568043, -0.08315136,  0.09149572,  0.01620813, -0.01365513,
         0.01828907, -0.07946642,  0.02152952,  0.04842284,  0.01012117,
        -0.11191086,  0.09578471,  0.05485644,  0.0005665 , -0.09153175],
       [ 0.02552603, -0.06210594,  0.12585063,  0.04519753, -0.07189874,
         0.07930857, -0.05440017,  0.01466599,  0.01500015, -0.02034149,
        -0.04936886,  0.08164561,  0.008405  ,  0.05134701, -0.03835429],
       [ 0.18454573,  0.01748664,  0.129515  ,  0.21854918, -0.27343598,
        -0.08594046,  0.08639184,  0.05960675, 

In [102]:
# 对tag的vector取平均
row_vectors = []
index = 0
for row in all_tags:
    length = len(row)
    vec = np.zeros(tags_trainX.shape[1])
    for i in range(length):
        vec = np.add(tags_trainX[index + i], vec)
    index += length
    row_vectors.append(vec / length)

In [103]:
row_vectors = np.array(row_vectors)

In [105]:
row_vectors.shape

(80000, 15)

In [104]:
row_vectors[:5]

array([[ 0.08484641, -0.02725166,  0.13304784,  0.04997884, -0.0568119 ,
         0.04010898, -0.00334538,  0.04008091,  0.04705598,  0.00537756,
        -0.04408284,  0.10696773,  0.04833685,  0.09938622, -0.08260272],
       [ 0.1043881 , -0.05153029,  0.16037631,  0.06658724, -0.11358675,
         0.00384869,  0.00907696,  0.03387623,  0.04921357,  0.02979287,
         0.00181418,  0.150112  ,  0.10527413,  0.05192956, -0.07227581],
       [ 0.0220023 , -0.01291083,  0.08266036, -0.01363504, -0.02485468,
         0.06361173, -0.01181254,  0.0163911 ,  0.02610213,  0.00560042,
        -0.03124853,  0.07094519,  0.03753572,  0.04144868, -0.05830737],
       [ 0.04568988, -0.06052803,  0.14369816,  0.01547725, -0.02400963,
        -0.00829319, -0.01311034,  0.04323305,  0.04154015,  0.01412414,
         0.00253889,  0.1079767 ,  0.06621586,  0.01378967, -0.05459015],
       [ 0.01228752, -0.06209987,  0.1395914 ,  0.01625125, -0.0168226 ,
         0.06358946, -0.05355548,  0.06568553, 

In [106]:
X_train = df.iloc[:].values
y_train = labels.values

In [107]:
X_train[:5]

array([[2.550e+02, 8.100e+00, 5.700e+01, 1.290e+03, 1.000e+01, 1.000e+00],
       [2.110e+02, 8.600e+00, 0.000e+00, 2.923e+03, 1.900e+01, 1.200e+01],
       [1.890e+02, 9.200e+00, 2.700e+01, 7.810e+02, 2.900e+01, 2.400e+01],
       [1.258e+03, 9.400e+00, 2.000e+01, 4.204e+03, 3.100e+01, 3.000e+00],
       [2.890e+02, 8.800e+00, 0.000e+00, 1.519e+03, 3.000e+00, 9.000e+00]])

In [110]:
X_train = np.hstack([X_train, row_vectors])

In [111]:
X_train[:5]

array([[ 2.55000000e+02,  8.10000000e+00,  5.70000000e+01,
         1.29000000e+03,  1.00000000e+01,  1.00000000e+00,
         8.48464084e-02, -2.72516580e-02,  1.33047839e-01,
         4.99788448e-02, -5.68118965e-02,  4.01089850e-02,
        -3.34537930e-03,  4.00809146e-02,  4.70559758e-02,
         5.37755930e-03, -4.40828386e-02,  1.06967730e-01,
         4.83368530e-02,  9.93862239e-02, -8.26027170e-02],
       [ 2.11000000e+02,  8.60000000e+00,  0.00000000e+00,
         2.92300000e+03,  1.90000000e+01,  1.20000000e+01,
         1.04388103e-01, -5.15302910e-02,  1.60376307e-01,
         6.65872410e-02, -1.13586751e-01,  3.84868532e-03,
         9.07695666e-03,  3.38762306e-02,  4.92135704e-02,
         2.97928691e-02,  1.81418178e-03,  1.50111999e-01,
         1.05274135e-01,  5.19295618e-02, -7.22758085e-02],
       [ 1.89000000e+02,  9.20000000e+00,  2.70000000e+01,
         7.81000000e+02,  2.90000000e+01,  2.40000000e+01,
         2.20022979e-02, -1.29108314e-02,  8.26603575e

In [112]:
import decision_tree.decision_tree
reload(decision_tree.decision_tree)
from decision_tree.decision_tree import RegressionTree

import gradient_boosting_decision_tree.gbdt_model
reload(gradient_boosting_decision_tree.gbdt_model)
from gradient_boosting_decision_tree.gbdt_model import GBDTRegressor

from sklearn.metrics import make_scorer

In [113]:
corrcoef = make_scorer(lambda y_true, y_pred: np.corrcoef(y_true, y_pred)[0,1]) 

def cross_val(x,y,n_estimators=100,learning_rate=0.15,max_depth=3):
    return cross_val_score(GBDTRegressor(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth),
                           x, y, scoring=corrcoef, cv=5).mean()

In [37]:
cross_val(X_train,y_train,n_estimators=100,learning_rate=0.15,max_depth=3)   # 无tag

0.648116130702245
0.6562377124733396
0.6516970866896995
0.6490815853956813
0.6630296823412862


0.6536324395204505

In [114]:
cross_val(X_train,y_train,n_estimators=100,learning_rate=0.15,max_depth=3)   # tag 15

0.6528607780146541

In [56]:
cross_val(X_train,y_train,n_estimators=100,learning_rate=0.15,max_depth=3)   # tag 10

0.6465490178790642
0.6544374048683615
0.6503916660277744


KeyboardInterrupt: 

In [ ]:
cross_val(X_train,y_train,n_estimators=100,learning_rate=0.15,max_depth=3)   # tag 5

0.642408055086571
